<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_Baraff_Witkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install meshzoo

In [ ]:
from collections import namedtuple
import numpy as np
import meshzoo
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
matplotlib.rc('animation', html='jshtml')

In [ ]:
vertex_positions_uv, triangle_indices = meshzoo.rectangle_tri(
    (0.0, 0.0),
    (1.0, 1.0),
    n=3,
    variant="zigzag",  # or "up", "down", "center"
)

print(vertex_positions_uv.shape)
print(triangle_indices.shape)

In [ ]:
n_vertices = vertex_positions_uv.shape[0]
vertex_positions_z = np.zeros(n_vertices)
vertex_positions = np.column_stack([vertex_positions_uv, vertex_positions_z])

In [ ]:
def plot_cloth(ax, vertex_positions, triangle_indices):
    x, y, z = vertex_positions.transpose()
    ax.clear()  # necessary for the animations
    ax.plot_trisurf(x, y, z, triangles=triangle_indices, color='deepskyblue')
    ax.scatter(x, y, z, c='deeppink', s=20, depthshade=False)
    for i, (xi, yi, zi) in enumerate(zip(x, y, z)):
        ax.text(xi, yi, zi, str(i), fontsize='large', color='black', zorder=10)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_zlim([-2, 0.25])

fig = plt.figure(figsize=(8, 6), dpi=100)
ax = fig.add_subplot(111, projection='3d')
plot_cloth(ax, vertex_positions, triangle_indices)

In [ ]:
def triangle_area(triangle_vertices):
    v0, v1, v2 = triangle_vertices
    return np.linalg.norm(np.cross(v1 - v0, v2 - v0)) / 2.0

def calculate_area(triangles_indices, vertex_positions):
    area = 0.0
    for triangle in triangle_indices:
        triangle_vertices = vertex_positions[triangle]
        area += triangle_area(triangle_vertices)
    return area

def calculate_vertex_masses(vertex_positions_uv, triangle_indices, density=0.2):
    vertex_masses = np.zeros(vertex_positions_uv.shape[0])

    for i,j,k in triangle_indices:
        area = triangle_area(vertex_positions_uv[[i, j, k]])
        triangle_mass = density * area
        vertex_masses[i] += 1/3 * triangle_mass 
        vertex_masses[j] += 1/3 * triangle_mass 
        vertex_masses[k] += 1/3 * triangle_mass

    return vertex_masses

def calculate_uv_stuff(triangle_vertex_positions_uv):
    uv_i, uv_j, uv_k = triangle_vertex_positions_uv
    ui, vi = uv_i
    uj, vj = uv_j
    uk, vk = uv_k

    delta_u1 = uj - ui
    delta_u2 = uk - ui
    delta_v1 = vj - vi
    delta_v2 = vk - vi

    delta_u_matrix = np.array([(delta_u1, delta_u2),
                               (delta_v1, delta_v2)])
    
    inverted_delta_u_matrix = np.linalg.inv(delta_u_matrix)

    dw_denominator = (delta_u1 * delta_v2 - delta_u2 * delta_v1)

    Xi_factor = (delta_v1 - delta_v2) / dw_denominator
    Xj_factor = delta_v2 / dw_denominator
    Xk_factor = -delta_v1 / dw_denominator

    return np.array([Xi_factor, Xj_factor, Xk_factor]), inverted_delta_u_matrix

def precompute_triangles_uv_stuff(vertex_positions_uv, triangle_indices):
    triangle_X_factors = []
    inverted_delta_u_matrices = []

    for triangle in triangle_indices:
        X_factors, inverted_delta_u_matrix = calculate_uv_stuff(vertex_positions_uv[triangle])
        triangle_X_factors.append(X_factors)
        inverted_delta_u_matrices.append(inverted_delta_u_matrix)
    
    return np.array(triangle_X_factors), np.array(inverted_delta_u_matrices)

In [ ]:
# Required attributes for a Baraff-Witkin style cloth simulator
class ClothSimulationMesh:
    def __init__(self, vertex_positions, vertex_positions_uv, triangle_indices):
        self.n_vertices = vertex_positions_uv.shape[0]
        self.n_triangles = triangle_indices.shape[0]

        # State 
        self.vertex_positions = vertex_positions.copy()
        self.vertex_velocities = np.zeros_like(vertex_positions)  

        # Configuration / Parameters
        self.triangle_indices = triangle_indices
        self.vertex_positions_uv = vertex_positions_uv
        self.vertex_masses = calculate_vertex_masses(vertex_positions_uv, triangle_indices)  
        self.triangle_stretch_stiffness_u = np.ones(self.n_triangles) 
        self.triangle_stretch_stiffness_v = np.ones(self.n_triangles)
        self.pinned_vertices = np.zeros(self.n_vertices) # could also be saved as a sparse attribute

        # Simulation (intermediate/cache) storage
        self.vertex_forces = np.zeros_like(self.vertex_velocities)
        self.precompute_uv_stuff()

    def precompute_uv_stuff(self):
        self.triangle_areas_uv = np.array([triangle_area(vertex_positions_uv[tri]) for tri in triangle_indices])
        triangle_X_factors, inverted_delta_u_matrices = precompute_triangles_uv_stuff(vertex_positions_uv, triangle_indices)
        self.triangle_X_factors = triangle_X_factors
        self.triangle_inverted_delta_u_matrices = inverted_delta_u_matrices   # (2, 2) matrix on triangle domain 

    def reset_vertex_forces(self):
        self.vertex_forces.fill(0)

In [ ]:
def calculate_forces(cloth):
    for triangle_index, triangle in enumerate(cloth.triangle_indices):
        i, j, k = triangle

        Xi, Xj, Xk = cloth.vertex_positions[triangle]

        delta_X1 = Xj - Xi
        delta_X2 = Xk - Xi

        inverted_delta_u_matrix = cloth.triangle_inverted_delta_u_matrices[triangle_index]
            
        delta_X_matrix = np.column_stack((delta_X1, delta_X2))

        # Equation (9) in Baraff-Witkin.
        w_uv = delta_X_matrix @ inverted_delta_u_matrix
        wu, wv = np.hsplit(w_uv, 2)     # wu = "world u"
        norm_wu = np.linalg.norm(wu)
        norm_wv = np.linalg.norm(wv)

        area_uv = cloth.triangle_areas_uv[triangle_index]

        Cu = area_uv * (norm_wu - 1.0)

        Xi_factor, Xj_factor, Xk_factor = cloth.triangle_X_factors[triangle_index]

        wu_normalized = wu / norm_wu

        dCu_Xi = area_uv * Xi_factor * wu_normalized  # it would be slightly more effecient to multiply all the scalars first and only then to the vector multiply
        dCu_Xj = area_uv * Xj_factor * wu_normalized
        dCu_Xk = area_uv * Xk_factor * wu_normalized

        u_stiffness = cloth.triangle_stretch_stiffness_u[triangle_index]
        v_stiffness = cloth.triangle_stretch_stiffness_v[triangle_index]

        # Equation (7) in Baraff-Witkin.
        force_i = u_stiffness * Cu * dCu_Xi
        force_j = u_stiffness * Cu * dCu_Xj
        force_k = u_stiffness * Cu * dCu_Xk

        cloth.vertex_forces[i] += force_i.reshape((3,)) # Reshape from (3, 1) to (3,)
        cloth.vertex_forces[j] += force_j.reshape((3,))
        cloth.vertex_forces[k] += force_k.reshape((3,))

# calculate_forces(cloth)

In [ ]:
# It would have been nice if these derivates where give in the original paper,
# sadly they weren't so I had to derive them myself by hand. I also found
# this blog online with the derivation, however I was confused a bit about the 
# notation at first, so I'll make my own blog post. 
# blog: http://davidpritchard.org/freecloth/docs/report-single/

# todo think about a lockfree way to fill force vector
# force vector has 1 element per vertex, but most vertices are contained in multiple triangles
# so if we compute the triangle forces in parallel, we need a lock when writing to the vertex position

# maybe: one pass over triangles -> local storage per triangle

# solution parallel pass over triangles -> store calculated forces locally
# then sequential pass over triangles that copies triangle forces into forces vector

# important abstraction later: function that fill in forces vector
# almost all sims work with forces applied to particles
# in BW98 via triangle based stretch condition (in Blender if mesh is passed with quads, just use quad face attribute on looptris)
# in MSD sims via edge based hooke's law

# also: function that fills force jacobians matrix

In [ ]:
def step(cloth, dt):
    # cloth.reset_vertex_forces()
    # cloth.vertex_forces += calculate_forces(cloth)

    vertex_accelerations = np.zeros_like(cloth.vertex_velocities)
    vertex_accelerations[:, -1] = -9.81 # acceleration due to gravity

    # cloth.vertex_accelerations += cloth.vertex_forces / cloth.vertex_masses

    cloth.vertex_velocities += vertex_accelerations * dt
    # print(cloth.vertex_velocities)

    cloth.vertex_positions += cloth.vertex_velocities * dt

In [ ]:
timesteps = 100         
dt = 0.01

cloth = ClothSimulationMesh(vertex_positions, vertex_positions_uv, triangle_indices)

def simulate(cloth, timesteps, dt):
    history = [cloth.vertex_positions.copy()]
    for _ in range(timesteps):
        step(cloth, dt)
        history.append(cloth.vertex_positions.copy())

    return history

history = simulate(cloth, timesteps, dt)

In [ ]:
history[0]

In [ ]:
def animate_cloth(history, dt, fps=50):
    fig = plt.figure(figsize=(5, 5), dpi=100)
    fig.subplots_adjust(0,0,1,1,0,0) # less padding
    ax = fig.add_subplot(111, projection='3d')
    plt.close()  # prevents duplicate output 

    fps_simulation = 1 / dt
    skip = np.floor(fps_simulation / fps).astype(np.int32)
    fps_adjusted = fps_simulation / skip
    print('fps was adjusted to:', fps_adjusted)

    def animate(i):
        j = min(i * skip, len(history) - 1)
        plot_cloth(ax, history[j], triangle_indices)
        ax.text2D(0.1, 0.9, 't = {:.3f}s'.format(j * dt), transform=ax.transAxes)


    n_frames = (len(history) - 1) // skip + 1
    interval = 1000*dt*skip
    anim = animation.FuncAnimation(fig, animate, frames=n_frames, interval=interval)
    return anim

animate_cloth(history, dt)